In [2]:
from app.db import collection_causas

In [4]:
document = {
                "id_causa": "C-2024-001",
                "titulo": "Demanda por incumplimiento de contrato",
                "descripcion": "El demandante alega falta de pago según contrato firmado el 15/03/2024",
                "estado": "en_tramite",
                "fecha_creacion": "2024-06-05T10:00:00Z",
                "fecha_ultima_actualizacion": "2024-06-10T15:30:00Z",
                "tipo": "civil",
                "clasificacion": "contratos",
                "partes": {
                    "demandante": {
                        "nombre": "Juan Pérez",
                        "rut": "12.345.678-9",
                        "abogado": "María González",
                        "contacto": "+56912345678"
                    },
                    "demandado": {
                        "nombre": "Empresa XYZ S.A.",
                        "rut": "76.543.210-K",
                        "abogado": "Pedro Sánchez"
                    }
                },
                "tribunal": {
                    "nombre": "Juzgado Civil de Santiago",
                    "rol_tribunal": "C-1245-2024"
                },
                "notas": "El demandado solicitó ampliación de plazo",
                "usuario_responsable": "abogado1@estudio.cl"
            }
id_causa = document.get("id_causa")
print(id_causa)

C-2024-001


In [5]:
collection_causas

AsyncCollection(AsyncDatabase(AsyncMongoClient(host='mongodb+srv://causas-demo.odphcgt.mongodb.net', document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='causas-demo'), 'causas-demo'), 'causas')

In [4]:
await collection_causas.update_one(
                        {"id_causa":id_causa},
                        {"$set":document},
                        upsert=True
                        )

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000118'), 'opTime': {'ts': Timestamp(1750082853, 4), 't': 280}, 'upserted': ObjectId('68502525b6060ecfd7ae4923'), 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1750082853, 4), 'signature': {'hash': b'\xce\x141\x05\x1bD\xc67\xd8\xe0\xd4B\n=\xab\x7f\xecT\x88\xa4', 'keyId': 7470607179121164290}}, 'operationTime': Timestamp(1750082853, 4), 'updatedExisting': False}, acknowledged=True)

In [6]:
c = await collection_causas.find_one({"id_causa":id_causa})

In [7]:
c

{'_id': ObjectId('68502525b6060ecfd7ae4923'),
 'id_causa': 'C-2024-001',
 'clasificacion': 'contratos',
 'descripcion': 'El demandante alega falta de pago según contrato firmado el 15/03/2024',
 'estado': 'en_tramite',
 'fecha_creacion': '2024-06-05T10:00:00Z',
 'fecha_ultima_actualizacion': '2024-06-10T15:30:00Z',
 'notas': 'El demandado solicitó ampliación de plazo',
 'partes': {'demandante': {'nombre': 'Juan Pérez',
   'rut': '12.345.678-9',
   'abogado': 'María González',
   'contacto': '+56912345678'},
  'demandado': {'nombre': 'Empresa XYZ S.A.',
   'rut': '76.543.210-K',
   'abogado': 'Pedro Sánchez'}},
 'tipo': 'civil',
 'titulo': 'Demanda por incumplimiento de contrato',
 'tribunal': {'nombre': 'Juzgado Civil de Santiago',
  'rol_tribunal': 'C-1245-2024'},
 'usuario_responsable': 'abogado1@estudio.cl'}

In [28]:
from datetime import datetime, timezone

In [34]:
b = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")
b

'2025-06-16T15:12:01.000Z'

In [35]:
a = datetime.now(timezone.utc).isoformat()
a

'2025-06-16T19:12:02.595200+00:00'

In [37]:
datetime.fromisoformat(b)

datetime.datetime(2025, 6, 16, 15, 12, 1, tzinfo=datetime.timezone.utc)

In [20]:
from typing import Literal
from app.db import collection_causas  # Ajusta el import según tu estructura

import re

async def get_causa_by_id(id_causa: str) -> dict | None:
    """
    Busca una causa judicial en la base de datos según su id_causa, validando el formato y corrigiendo la capitalización.

    Parámetros:
    -----------
    id_causa : str
        El identificador único de la causa (ejemplo: "C-2025-001").
        Debe cumplir el formato: C-YYYY-NNN (ejemplo: "C-2025-001").
        Si el usuario ingresa la letra inicial en minúscula, la función la corrige automáticamente.

    Retorna:
    --------
    dict | None
        Si el formato es incorrecto, retorna {'error': 'Formato de id_causa inválido'}.
        Si encuentra la causa, retorna el dict con todos los datos de la causa encontrada.
        Si no encuentra ninguna causa, retorna None.

    Ejemplo de uso:
    --------------
    causa = await get_causa_by_id("c-2025-001")
    # Corrige a "C-2025-001" y busca correctamente.
    """
    # Corrige la capitalización del primer carácter
    if id_causa and id_causa[0].islower():
        id_causa = id_causa[0].upper() + id_causa[1:]

    pattern = r'^C-\d{4}-\d{3}$'
    if not re.match(pattern, id_causa):
        return {'error': 'Formato de id_causa inválido. Debe ser C-YYYY-NNN (ej: C-2025-001)'}

    filtro = {'id_causa': id_causa}
    causa = await collection_causas.find_one(filtro, projection={'_id': 0})
    return causa

In [23]:
await get_causa_by_id(id_causa="e-2025-001")

{'error': 'Formato de id_causa inválido. Debe ser C-YYYY-NNN (ej: C-2025-001)'}